# Introdução

Este notebook visa apresentar o desenvolvimento realizado pelo Grupo 3 - composto por Tiago Sousa (20735) , Rodrigo Castro (23143), Rogério Gomes (27216), Paulo Costa (29851) e Laís Carvalho (51067) - no âmbito do trabalho prático “Project 01” da disciplina de Inteligêncial Artificial.

O projeto tem como objetivo a criação de um agente inteligente capaz de funcionar como gestor de horários de aulas para cursos de graduação numa instituição de ensino superior, considerando todas as restrições e condições que possam existir no processo de alocação de horas.

O problema de agendamento é resolvido utilizando programção de restrições (CSP).


In [38]:
# Install contraint library
!pip install python-constraint

# Import contraint library
from constraint import *


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Variáveis e Domínios

Começamos por definir as Variáveis e os seus respetivos Domínios tendo em conta o contexto do problema, seguindo rigorosamente as especificações do ficheiro de dados disponibilizado pelo professor.

As variáveis representam as alocações de tempo para cada aula. Cada variável é identificada pelo formato **ucXX_tYY_aulaZZ**, onde:
- ucXX: Representa a unidade curricular;
- tYY: Identifica a Turma;
- aulaZZ: Indica se é a primeira ou segunda sessão de aula da respetiva UC;

Dado que todas as Ucs possuem duas aulas por semana, o problema é definido por um total de 30 variáveis.

<br>
Os domínios correspondem ao conjunto de valores possíveis para cada variável.

Foi feita uma filtração inicial para respeitar as indisponibilidades iniciais fornecidas nas regras **#tr**, **#rc** e **#oc** do ficheiro.

In [39]:
from constraint import *
from itertools import chain

# instatiate the problem
problem = Problem()

# add variables
problem.addVariable('uc11_t01_aula01', range(1,20))
problem.addVariable('uc11_t01_aula02', range(1,20))

problem.addVariable('uc12_t01_aula01', range(1,12))
problem.addVariable('uc12_t01_aula02', range(1,12))

problem.addVariable('uc13_t01_aula01', range(5,20))
problem.addVariable('uc13_t01_aula02', range(5,20))

problem.addVariable('uc14_t01_aula01', range(5,20))
problem.addVariable('uc14_t01_aula02', range(5,20))

problem.addVariable('uc15_t01_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc15_t01_aula02', list(chain(range(1, 9), range(13, 17))))

problem.addVariable('uc21_t02_aula01', range(1,20))
problem.addVariable('uc21_t02_aula02', range(1,20))

problem.addVariable('uc22_t02_aula01', range(1,20))
problem.addVariable('uc22_t02_aula02', range(1,20))

problem.addVariable('uc23_t02_aula01', range(1,12))
problem.addVariable('uc23_t02_aula02', range(1,12))

problem.addVariable('uc24_t02_aula01', range(5,20))
problem.addVariable('uc24_t02_aula02', range(5,20))

problem.addVariable('uc25_t02_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc25_t02_aula02', list(chain(range(1, 9), range(13, 17))))

problem.addVariable('uc31_t03_aula01', range(1,20))
problem.addVariable('uc31_t03_aula02', range(1,20))

problem.addVariable('uc32_t03_aula01', range(1,12))
problem.addVariable('uc32_t03_aula02', range(1,12))

problem.addVariable('uc33_t03_aula01', range(5,20))
problem.addVariable('uc33_t03_aula02', range(5,20))

problem.addVariable('uc34_t03_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc34_t03_aula02', list(chain(range(1, 9), range(13, 17))))

problem.addVariable('uc35_t03_aula01', list(chain(range(1, 9), range(13, 17))))
problem.addVariable('uc35_t03_aula02', list(chain(range(1, 9), range(13, 17))))




# Restrições
O sistema de agendamento depende da definição de Restrições, que são cruciais para garantir a validade do horário. 

## Funções Auxiliares
Estas funções permitem a conversão dos espaços númericos (1-20) em valores lógicos de **Dia** (1-5) e **Bloco** (1-4), facilitando a formulação precisa de todas as restrições baseadas em tempo.

Vão ser usadas para ajudar a definir as funções de restrição.

In [40]:
from constraint import *
from itertools import chain

def get_day(slot):
    """Retorna o dia correspondente (1–5) para o slot (1–20)."""
    return (slot - 1) // 4 + 1

def get_block(slot):
    """Retorna o bloco (1–4) dentro do dia."""
    return (slot - 1) % 4 + 1

## Hard Constraints

O passo seguinte foi definir as condições que têm de ser satisfeitas para que o horário seja viável, as chamadas Hard Constraints. 

<br>

**Restrição 1** - Professor não pode ter 2 aulas ao mesmo tempo

In [41]:
teacher_courses = {
    "jo":  ["uc11_t01", "uc21_t02", "uc22_t02", "uc31_t03"],
    "mike":["uc12_t01", "uc23_t02", "uc32_t03"],
    "rob": ["uc13_t01", "uc14_t01", "uc24_t02", "uc33_t03"],
    "sue": ["uc15_t01", "uc25_t02", "uc34_t03", "uc35_t03"]
}


COURSE_TEACHER = {}
for t, cursos in teacher_courses.items():
    for c in cursos:
        COURSE_TEACHER[c] = t

def teacher_of(var):
    """Devolve o nome do professor que dá a UC associada à variável."""
    base = var.rsplit("_", 1)[0]  # ex.: 'uc11_t01'
    return COURSE_TEACHER.get(base, "—")

def teacher_conflict(a, b):
    return a != b  # não pode ter o mesmo slot

for teacher, cursos in teacher_courses.items():
    for i in range(len(cursos)):
        for j in range(i + 1, len(cursos)):
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula01", f"{cursos[j]}_aula01"])
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula01", f"{cursos[j]}_aula02"])
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula02", f"{cursos[j]}_aula01"])
            problem.addConstraint(teacher_conflict, [f"{cursos[i]}_aula02", f"{cursos[j]}_aula02"])



**Restrição 2** - Duas aulas do mesmo curso não podem estar no mesmo slot

In [42]:

def different_times(a, b):
    return a != b

for c in [
    "uc11_t01", "uc12_t01", "uc13_t01", "uc14_t01", "uc15_t01",
    "uc21_t02", "uc22_t02", "uc23_t02", "uc24_t02", "uc25_t02",
    "uc31_t03", "uc32_t03", "uc33_t03", "uc34_t03", "uc35_t03"
]:
    problem.addConstraint(different_times, [f"{c}_aula01", f"{c}_aula02"])



**Restrição 3** - Duas aulas da mesma cadeira devem agendadas em dias diferentes

In [43]:

def different_days(a, b):
    return get_day(a) != get_day(b)

for c in [
    "uc11_t01", "uc12_t01", "uc13_t01", "uc14_t01", "uc15_t01",
    "uc21_t02", "uc22_t02", "uc23_t02", "uc24_t02", "uc25_t02",
    "uc31_t03", "uc32_t03", "uc33_t03", "uc34_t03", "uc35_t03"
]:
    problem.addConstraint(different_days, [f"{c}_aula01", f"{c}_aula02"])



**Restrição 4** - Cada turma deve ter no máximo 4 dias de aulas distribuidas

In [44]:

turmas = {
    "t01": ["uc11_t01", "uc12_t01", "uc13_t01", "uc14_t01", "uc15_t01"],
    "t02": ["uc21_t02", "uc22_t02", "uc23_t02", "uc24_t02", "uc25_t02"],
    "t03": ["uc31_t03", "uc32_t03", "uc33_t03", "uc34_t03", "uc35_t03"]
}

# Nov
def exactly_four_days(*args):
    dias = {get_day(x) for x in args}
    return len(dias) == 4

for t, cursos in turmas.items():
    all_lessons = [f"{c}_aula01" for c in cursos] + [f"{c}_aula02" for c in cursos]
    problem.addConstraint(exactly_four_days, all_lessons)




**Restrição 5** - As aulas da mesma turma no mesmo dia devem ser em blocos consecutivos

In [45]:

def consecutive_blocks(*args):
    aulas_por_dia = {}
    for a in args:
        dia = get_day(a)
        bloco = get_block(a)
        aulas_por_dia.setdefault(dia, []).append(bloco)
    for blocos in aulas_por_dia.values():
        blocos.sort()
        for i in range(1, len(blocos)):
            if blocos[i] - blocos[i - 1] > 1:
                return False
    return True

for t, cursos in turmas.items():
    all_lessons = [f"{c}_aula01" for c in cursos] + [f"{c}_aula02" for c in cursos]
    problem.addConstraint(consecutive_blocks, all_lessons)

# bloqueia que duas disciplinas da mesma turma ocupem o mesmo slot
def not_same_slot(a, b):
    return a != b

# para cada par de cursos da turma t01, por exemplo:
turma_t01 = ["uc11_t01", "uc12_t01", "uc13_t01", "uc14_t01", "uc15_t01"]
for i in range(len(turma_t01)):
    for j in range(i+1, len(turma_t01)):
        problem.addConstraint(not_same_slot, [f"{turma_t01[i]}_aula01", f"{turma_t01[j]}_aula01"])
        problem.addConstraint(not_same_slot, [f"{turma_t01[i]}_aula01", f"{turma_t01[j]}_aula02"])
        problem.addConstraint(not_same_slot, [f"{turma_t01[i]}_aula02", f"{turma_t01[j]}_aula01"])
        problem.addConstraint(not_same_slot, [f"{turma_t01[i]}_aula02", f"{turma_t01[j]}_aula02"])

# Cada turma não pode ter duas aulas no mesmo slot (AllDifferentConstraint)
from constraint import AllDifferentConstraint

for t, cursos in turmas.items():
    all_lessons = [f"{c}_aula01" for c in cursos] + [f"{c}_aula02" for c in cursos]
    problem.addConstraint(AllDifferentConstraint(), all_lessons)



## Separação das aulas em salas obrigatórias das aulas com salas fléxiveis

In [46]:
aulas_lab01 = [
    "uc14_t01_aula01", "uc14_t01_aula02",
    "uc22_t02_aula01", "uc22_t02_aula02"
]

aulas_online = [
    "uc21_t02_aula02", "uc21_t02_aula01",
    "uc31_t03_aula02", "uc31_t03_aula01"
]

# todas as variáveis do problema
all_variables = list(problem._variables.keys())


aulas_fixas = aulas_lab01 + aulas_online
aulas_livres = [v for v in all_variables if v not in aulas_fixas]


**Restrição 6** - Todas as aulas com sala obrigatória não podem ser agendadas no mesmo slot de tempo

In [47]:

def room_conflict(a, b):
    """ Garante que duas aulas não ocupam o mesmo slot na mesma sala."""
    return a != b

# conflito LAB01
for i in range(len(aulas_lab01)):
    for j in range(i + 1, len(aulas_lab01)):
        problem.addConstraint(room_conflict, [aulas_lab01[i], aulas_lab01[j]])

# conflito Online
if len(aulas_online) > 1:
    problem.addConstraint(room_conflict, [aulas_online[0], aulas_online[1]])


**Restrição 7** - O número de aulas agendadas em qualquer slot de tempo não pode exceder o número total de salas disponíveis (3)

In [48]:
def max_rooms_constraint(*slots):
    MAX_LIVRES = 3
    slot_counts = {}
    
    for slot in slots:
        slot_counts[slot] = slot_counts.get(slot, 0) + 1
        
    for count in slot_counts.values():
        if count > MAX_LIVRES: 
            return False
    return True


problem.addConstraint(max_rooms_constraint, aulas_livres)

## Soft Constraints - Heurísticas de avaliação
Com todas as Hard Constraints satisfeitas, o nosso foco passou para a otimização da qualidade do horário. Esta otimização é alcançada através do uso de Soft Constraints e da Greedy Search.

<br>

Esta função atua como a heurística principal da Greedy Search, fornecendo a informação necessária para guiar a pesquisa e fazer a melhor escolha localmente.

O seu objetivo é atribuir um score a cada slot que está a ser testado. Este score é a métrica de qualidade que o algoritmo irá tentar maximizar, através dos critérios de organização definidos (Soft Constraints)




In [49]:
def score_soft_constraints(solution, course, slot):
    """Calcula score para soft constraints com base na atribuição parcial."""
    score = 0

    # (1) Dias distintos para aulas do mesmo curso
    if course in solution:
        dia_outro = (solution[course] - 1)//4 + 1
        dia_atual = (slot - 1)//4 + 1
        if dia_outro != dia_atual:
            score += 5

    # (2) Consecutividade das aulas de uma turma
    turma = course.split("_")[1]  # ex: 't01'
    blocos_por_dia = {}
    for var, s in solution.items():
        if turma in var:
            dia = (s-1)//4 + 1
            bloco = (s-1)%4 + 1
            blocos_por_dia.setdefault(dia, []).append(bloco)
    # adiciona o slot candidato
    dia = (slot-1)//4 + 1
    bloco = (slot-1)%4 + 1
    blocos_por_dia.setdefault(dia, []).append(bloco)
    for blocos in blocos_por_dia.values():
        blocos.sort()
        for i in range(1, len(blocos)):
            if blocos[i] - blocos[i-1] == 1:
                score += 2  # recompensa blocos consecutivos

    # (3) Limite de 4 dias por turma
    dias = set()
    for var, s in solution.items():
        if turma in var:
            dias.add((s-1)//4 + 1)
    dias.add((slot-1)//4 + 1)
    
    
    if len(dias) == 4:
        score += 2
    elif len(dias) > 4:
        score -= 5 

    return score

# Validação das Hard Constraints na Greedy Search
def curso_id(var):           # 'uc11_t01_aula01' -> 'uc11_t01'
    return var.rsplit('_', 1)[0]

def turma_of_var(var):        # 'uc11_t01_aula01' -> 't01'
    return var.split('_')[1]

# mapa rápido: curso base -> professor
COURSE_TEACHER = {}
for t, cursos in teacher_courses.items():
    for c in cursos:
        COURSE_TEACHER[c] = t

def prof_of_var(var):
    return COURSE_TEACHER.get(curso_id(var), None)



def hard_ok(partial, var, slot):

    turma = turma_of_var(var)
    prof  = prof_of_var(var)
    dslot = get_day(slot)

    # (4) máx. 4 dias por turma (EXISTENTE)
    dias_usados = { get_day(s) for w, s in partial.items() if turma_of_var(w) == turma }
    dias_usados.add(dslot)
    if len(dias_usados) > 4:
        return False
        
   
   # (6) máx. 1 aula no LAB01 por slot
    if var in aulas_lab01:
        for w, s in partial.items():
            if w in aulas_lab01 and s == slot:
                return False  # Conflito no LAB01

    # (6) máx. 1 aula online por slot
    if var in aulas_online:
        for w, s in partial.items():
            if w in aulas_online and s == slot:
                return False # Conflito na Sala Online

    # (7) conflito das aulas
    if var in aulas_livres:
        MAX_LIVRES = 3
        count_livres = 1 
        for w, s in partial.items():
            # Conta as aulas livres já agendadas para este slot
            if w in aulas_livres and s == slot:
                count_livres += 1
        
        if count_livres > MAX_LIVRES:
            return False 
            
   
    for w, s in partial.items():
        # (2) AllDifferent por turma
        if turma_of_var(w) == turma and s == slot:
            return False
        # (3) professor same slot
        if prof_of_var(w) == prof and s == slot:
            return False
        # (1) mesma UC: slots != e dias !=
        if curso_id(w) == curso_id(var):
            if s == slot:
                return False
            if get_day(s) == dslot:
                return False

    return True



## Greedy Search
O Algoritmo Greedy Search utiliza a pontuação feita como critério de escolha. 

A estratégia é fazer a melhor escolha local (a que dá o maior score) em cada passo do agendamento, garantindo que o slot escolhido: 
1) é válido - respeita as Hard Constraints  
2) Maximiza a pontuação das Soft Constraints.

In [50]:
# === Greedy Search ===
def greedy_solution(problem, courses):
    # Heurísticas MRV (prioriza variáveis com menos opções)
    def degree(v):
        deg = 0
        tv = turma_of_var(v)
        pv = prof_of_var(v)
        for w in problem._variables.keys():
            if w == v: 
                continue
            if turma_of_var(w) == tv:
                deg += 1
            if prof_of_var(w) == pv:
                deg += 1
            if curso_id(w) == curso_id(v):
                deg += 1
        return deg

    vars_order = list(courses)
    vars_order.sort(key=lambda v: (len(list(problem._variables[v])), -degree(v)))

    solution = {}
    for var in vars_order:
        # só slots que não violam hard constraints
        candidates = [s for s in problem._variables[var] if hard_ok(solution, var, s)]
        if not candidates:
            raise RuntimeError(f"Sem slots válidos para {var} dadas as HARD constraints.")

     
        candidates.sort(key=lambda s: score_soft_constraints(solution, var, s), reverse=True)

        solution[var] = candidates[0]

    return solution





# Solução do problema

Esta seção final apresenta a solução para o problema de agendamento, demonstrando o resultado otimizado.

Inicialmente, é feita uma definição dos critérios de avaliação para se perceber a otimização da solução.

Maior Score = Mais Otimizado de acordo com as Soft Constraints

In [51]:
# FUNÇÕES DE AVALIAÇÃO (SOFT CONSTRAINTS) 

def aulas_consecutivas(solution, turma):
    """Conta quantos pares de aulas consecutivas uma turma tem."""
    blocos_por_dia = {}
    for var, slot in solution.items():
        if turma in var:
            dia = (slot - 1)//4 + 1
            bloco = (slot - 1)%4 + 1
            blocos_por_dia.setdefault(dia, []).append(bloco)
    consecutivas = 0
    for blocos in blocos_por_dia.values():
        blocos.sort()
        for i in range(1, len(blocos)):
            if blocos[i] - blocos[i - 1] == 1:
                consecutivas += 1
    return consecutivas

def dias_distintos(solution, turma):
    """Conta quantos dias uma turma tem aulas."""
    dias = set()
    for var, slot in solution.items():
        if turma in var:
            dias.add((slot - 1)//4 + 1)
    return len(dias)

def avaliar_solucao(solution):
    """Avalia a solução com base nas soft constraints."""
    
    score = 0
    
    for turma in ["t01", "t02", "t03"]:
        score += aulas_consecutivas(solution, turma) * 2
        dias = dias_distintos(solution, turma)
        if dias <= 4:
            score += 3  
        else:
            score -= 50
    return score

def get_full_room_assignment(solution):
    """Atribuição das salas para cada aula na solução."""
    room_map = {}
    NOMES_SALAS_LIVRES = ["Sala A", "Sala B", "Sala C"]
    
 
    salas_livres_ocupadas_por_slot = {} 
    
   
    for var, slot in sorted(solution.items()):
        
       
        if var in aulas_lab01:
            sala = "Lab01"
        elif var in aulas_online:
            sala = "Online"
            
     
        elif var in aulas_livres:
            salas_livres_ocupadas_por_slot.setdefault(slot, [])
            
            # O índice de ocupação (0, 1, 2) corresponde à Sala A, B, C
            indice_de_ocupacao = len(salas_livres_ocupadas_por_slot[slot])
            
            if indice_de_ocupacao < len(NOMES_SALAS_LIVRES): 
                sala = NOMES_SALAS_LIVRES[indice_de_ocupacao] 
            else:
                
                sala = "ERRO: Capacidade Excedida" 
            
            #regista aula no slot livre
            salas_livres_ocupadas_por_slot[slot].append(var)
        else:
            sala = "N/D"

        room_map[var] = sala
        
    return room_map

Esta função mostra o horário tabelado.

In [52]:
def mostrar_horarios_por_turma_sem_tabulate(solution_base):
    """
    Mostra o horário de cada turma em formato de tabela (sem dependências externas).
    Usa teacher_of(var) e room_assignment_map[var].
    Cada dia tem 4 aulas (2 manhã e 2 tarde).
    """
    turmas = {}
    
    # Organiza por turma
    for var, slot in sorted(solution_base.items()):
        partes = var.split('_')
        if len(partes) < 3:
            continue

        uc = partes[0].upper()     # ex: UC11
        turma = partes[1].upper()  # ex: T01
        aula = partes[2].upper()   # ex: AULA01
        prof = teacher_of(var)
        sala = room_assignment_map.get(var, "—")

        try:
            slot = int(slot)
        except Exception:
            slot = None

        turmas.setdefault(turma, []).append((uc, aula, sala, prof, slot))

    # Converte número → dia e hora
    def converter_para_dia_hora(n):
        dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
        if not isinstance(n, int) or n < 1:
            return "?"
        dia = dias[(n - 1) // 4] if 1 <= n <= 20 else "?"
        slot = (n - 1) % 4 + 1
        horas = {1: "08:00", 2: "09:00", 3: "14:00", 4: "15:00"}
        return f"{dia} {horas.get(slot, '?')}"

    # Impressão formatada (tabela simples)
    def imprimir_tabela(titulo, rows, headers=("UC", "Horário", "Sala", "Professor")):
        if not rows:
            print(f"\n=== Horário da {titulo} ===\n(Sem aulas)")
            return

        cols = list(zip(*([headers] + rows)))
        widths = [max(len(str(x)) for x in c) for c in cols]
        sep = "+" + "+".join("-" * (w + 2) for w in widths) + "+"

        print(f"\n=== Horário da {titulo} ===")
        print(sep)
        print("| " + " | ".join(str(h).ljust(w) for h, w in zip(headers, widths)) + " |")
        print(sep.replace("-", "="))
        for r in rows:
            print("| " + " | ".join(str(c).ljust(w) for c, w in zip(r, widths)) + " |")
        print(sep)

    # Montar e imprimir tudo
    for turma, aulas in sorted(turmas.items()):
        aulas.sort(key=lambda x: (x[4] is None, x[4] if x[4] is not None else 999))
        tabela = [
            (uc, converter_para_dia_hora(slot), sala, prof)
            for uc, aula, sala, prof, slot in aulas
        ]
        imprimir_tabela(turma, tabela)


De seguida, é feita uma pesquisa com backtracking que é utilizada para confirmar a existência de uma solução válida que satisfaz as Hard Constraints e as Soft Constraints.


In [53]:
# Execução de Pesquisa Backtracking 
try:
    solution_base = problem.getSolution()

    if solution_base:
        
        room_assignment_map = get_full_room_assignment(solution_base)
        mostrar_horarios_por_turma_sem_tabulate(solution_base)
        
       
        
        
        for var, slot in sorted(solution_base.items()):
            dia = (slot - 1)//4 + 1
            bloco = (slot - 1) % 4 + 1
            prof = teacher_of(var) 
        
            sala = room_assignment_map[var]
  
            print(f"{var}: Dia {dia}, Bloco {bloco}, Sala: {sala}, Professor: {prof}")

        score_base = avaliar_solucao(solution_base)
        print(f"\nHeuristic Score (Soft Constraints) Base: {score_base}")
    else:
        print("\nSolução Padrão: Não foi encontrada nenhuma solução válida.")

except Exception as e:
    print(f"\nErro na Solução Padrão: {e}")


=== Horário da T01 ===
+------+--------------+--------+-----------+
| UC   | Horário      | Sala   | Professor |
+======+==============+========+===========+
| UC12 | Terça 14:00  | Sala A | mike      |
| UC15 | Terça 15:00  | Sala A | sue       |
| UC12 | Quarta 14:00 | Sala A | mike      |
| UC11 | Quinta 08:00 | Sala A | jo        |
| UC13 | Quinta 09:00 | Sala A | rob       |
| UC15 | Quinta 14:00 | Sala A | sue       |
| UC14 | Quinta 15:00 | Lab01  | rob       |
| UC11 | Sexta 08:00  | Sala A | jo        |
| UC13 | Sexta 09:00  | Sala A | rob       |
| UC14 | Sexta 14:00  | Lab01  | rob       |
+------+--------------+--------+-----------+

=== Horário da T02 ===
+------+--------------+--------+-----------+
| UC   | Horário      | Sala   | Professor |
+======+==============+========+===========+
| UC25 | Terça 14:00  | Sala B | sue       |
| UC23 | Terça 15:00  | Sala B | mike      |
| UC23 | Quarta 08:00 | Sala A | mike      |
| UC24 | Quinta 08:00 | Sala B | rob       |
| UC21 

Por fim, o foco passa para o Algoritmo da Greedy Search. 

A solução final dada pela Greedy Search demonstra o resultado após o agente inteligente ter satisfeito todas as Hard Constraints e ter otimizado o horário com base nas Soft Constraints.

In [54]:
# Execução da Greedy Search

courses = list(problem._variables.keys())

try:

    solution_greedy = greedy_solution(problem, courses)

    print("\n--- Solução Greedy Search ---")
    
    mostrar_horarios_por_turma_sem_tabulate(solution_greedy)
    room_assignment_map = get_full_room_assignment(solution_greedy)
    
    
    for var, slot in sorted(solution_greedy.items()):
        dia = (slot - 1)//4 + 1
        bloco = (slot - 1) % 4 + 1
        prof = teacher_of(var) 
        sala = room_assignment_map[var]
        print(f"{var}: Dia {dia}, Bloco {bloco}, Sala: {sala}, Professor: {prof}")


    score_greedy = avaliar_solucao(solution_greedy)
    print(f"\nHeuristic Score (Soft Constraints): {score_greedy}")

except RuntimeError as e:
    print(f"\n Greedy não conseguiu atribuir todas as aulas: {e}")



--- Solução Greedy Search ---

=== Horário da T01 ===
+------+---------------+--------+-----------+
| UC   | Horário       | Sala   | Professor |
+======+===============+========+===========+
| UC11 | Segunda 09:00 | Sala A | jo        |
| UC12 | Segunda 14:00 | Sala A | mike      |
| UC15 | Segunda 15:00 | Sala A | sue       |
| UC13 | Terça 08:00   | Sala A | rob       |
| UC15 | Terça 09:00   | Sala A | sue       |
| UC12 | Terça 14:00   | Sala A | mike      |
| UC13 | Quarta 14:00  | Sala A | rob       |
| UC14 | Quarta 15:00  | Lab01  | rob       |
| UC11 | Quinta 08:00  | Sala A | jo        |
| UC14 | Quinta 09:00  | Lab01  | rob       |
+------+---------------+--------+-----------+

=== Horário da T02 ===
+------+---------------+--------+-----------+
| UC   | Horário       | Sala   | Professor |
+======+===============+========+===========+
| UC25 | Segunda 08:00 | Sala A | sue       |
| UC23 | Segunda 09:00 | Sala A | mike      |
| UC21 | Segunda 14:00 | Online | jo        |
|

##  Métricas e Comparação

Nesta secção medimos tempos, contamos todas as soluções possíveis num determinado período de tempo e escolhemos a melhor solução com base no **score** heurístico para depois ser comparada com a heurística Greedy (com validação de *hard constraints*).

In [55]:
# 1) Tempo para a primeira solução do solver
import time

def pretty(sol, title):
    print(f"\n{title}")
    for var in sorted(sol):
        slot = sol[var]
        dia = (slot - 1)//4 + 1
        bloco = (slot - 1)%4 + 1
        prof = teacher_of(var)
        print(f"{var}: Dia {dia}, Bloco {bloco}, Professor: {prof}")

t0 = time.perf_counter()
solution_first = problem.getSolution()
t1 = time.perf_counter()
print(f"⏱️ Solver (primeira solução) — {t1 - t0:.3f}s")
mostrar_horarios_por_turma_sem_tabulate(solution_first)
pretty(solution_first, "Primeira solução do solver")
print("Heuristic Score:", avaliar_solucao(solution_first))

⏱️ Solver (primeira solução) — 0.001s

=== Horário da T01 ===
+------+--------------+--------+-----------+
| UC   | Horário      | Sala   | Professor |
+======+==============+========+===========+
| UC12 | Terça 14:00  | Sala A | mike      |
| UC15 | Terça 15:00  | Sala A | sue       |
| UC12 | Quarta 14:00 | Sala A | mike      |
| UC11 | Quinta 08:00 | Sala A | jo        |
| UC13 | Quinta 09:00 | Sala A | rob       |
| UC15 | Quinta 14:00 | Sala A | sue       |
| UC14 | Quinta 15:00 | Lab01  | rob       |
| UC11 | Sexta 08:00  | Sala A | jo        |
| UC13 | Sexta 09:00  | Sala A | rob       |
| UC14 | Sexta 14:00  | Lab01  | rob       |
+------+--------------+--------+-----------+

=== Horário da T02 ===
+------+--------------+--------+-----------+
| UC   | Horário      | Sala   | Professor |
+======+==============+========+===========+
| UC25 | Terça 14:00  | Sala B | sue       |
| UC23 | Terça 15:00  | Sala B | mike      |
| UC23 | Quarta 08:00 | Sala B | mike      |
| UC24 | Quint

In [56]:
# 2) Contar soluções (até a um limite temporal) e escolher a melhor pelo score
best_solution = None
best_score = float("-inf")
count = 0

t0 = time.perf_counter()
time_limit_sec = 240  # podes ajustar
for s in problem.getSolutionIter():
    count += 1
    sc = avaliar_solucao(s)
    if sc > best_score:
        best_score = sc
        best_solution = s
    if time.perf_counter() - t0 > time_limit_sec:
        print(f"Parado por time_limit ({time_limit_sec}s).")
        break
t1 = time.perf_counter()

print(f"\nNúmero de soluções encontradas (até ao limite): {count}")
print(f"Tempo total: {t1 - t0:.3f}s")
print(f"Melhor score: {best_score}")
mostrar_horarios_por_turma_sem_tabulate(best_solution)
pretty(best_solution, "Melhor solução (pelo score)")

Parado por time_limit (240s).

Número de soluções encontradas (até ao limite): 1553572
Tempo total: 240.000s
Melhor score: 45

=== Horário da T01 ===
+------+--------------+--------+-----------+
| UC   | Horário      | Sala   | Professor |
+======+==============+========+===========+
| UC12 | Terça 14:00  | Sala A | mike      |
| UC15 | Terça 15:00  | Sala A | sue       |
| UC12 | Quarta 14:00 | Sala A | mike      |
| UC11 | Quinta 08:00 | Sala A | jo        |
| UC13 | Quinta 09:00 | Sala A | rob       |
| UC15 | Quinta 14:00 | Sala A | sue       |
| UC14 | Quinta 15:00 | Lab01  | rob       |
| UC11 | Sexta 08:00  | Sala A | jo        |
| UC13 | Sexta 09:00  | Sala A | rob       |
| UC14 | Sexta 14:00  | Lab01  | rob       |
+------+--------------+--------+-----------+

=== Horário da T02 ===
+------+--------------+--------+-----------+
| UC   | Horário      | Sala   | Professor |
+======+==============+========+===========+
| UC25 | Terça 14:00  | Sala B | sue       |
| UC23 | Terça 1

In [57]:
# 3) Tempo e score da heurística Greedy (HARD-safe)
courses = list(problem._variables.keys())
t0 = time.perf_counter()
greedy_sol = greedy_solution(problem, courses)
t1 = time.perf_counter()

print(f"\n⏱️ Greedy (tempo): {t1 - t0:.3f}s")
mostrar_horarios_por_turma_sem_tabulate(greedy_sol)
pretty(greedy_sol, "Solução da greedy (HARD respeitado)")
print("Heuristic Score:", avaliar_solucao(greedy_sol))


⏱️ Greedy (tempo): 0.003s

=== Horário da T01 ===
+------+---------------+--------+-----------+
| UC   | Horário       | Sala   | Professor |
+======+===============+========+===========+
| UC12 | Segunda 08:00 | Sala A | mike      |
| UC15 | Segunda 09:00 | Sala A | sue       |
| UC11 | Segunda 14:00 | Sala A | jo        |
| UC14 | Terça 08:00   | Lab01  | rob       |
| UC13 | Terça 09:00   | Sala A | rob       |
| UC12 | Terça 14:00   | Sala A | mike      |
| UC15 | Terça 15:00   | Sala A | sue       |
| UC11 | Quarta 09:00  | Sala A | jo        |
| UC13 | Quarta 14:00  | Sala A | rob       |
| UC14 | Quarta 15:00  | Lab01  | rob       |
+------+---------------+--------+-----------+

=== Horário da T02 ===
+------+---------------+--------+-----------+
| UC   | Horário       | Sala   | Professor |
+======+===============+========+===========+
| UC21 | Segunda 08:00 | Online | jo        |
| UC23 | Segunda 09:00 | Sala B | mike      |
| UC25 | Segunda 14:00 | Sala B | sue       |
| UC2

## Análise crítica dos resultados

**Abordagens testadas**
- Solver `python-constraint` (busca com hard constraints).
- Heurística Greedy com validação de *hard constraints* (UC em dias diferentes, sem choques por turma/professor, ≤4 dias por turma).

**Tempos de execução**
- Reportados acima para: primeira solução do solver, contagem de soluções até limite temporal e greedy.

**Qualidade das soluções**
- O *score* heurístico compara compactação (aulas consecutivas), distribuição (≤4 dias por turma) e outros critérios definidos.

**Observações**
- A restrição `AllDifferent` por turma combinada com regras globais (ex.: blocos consecutivos) aumenta o tempo do solver.
- Limitar previamente os dias por turma (filtro de domínio) e quebrar simetrias (`aula01 < aula02`) melhora desempenho.


## Melhorias futuras do agente

- **Heurísticas de procura**: MRV/LCV + *forward checking* para acelerar a seleção.
- **Quebra de simetria**: impor `aula01 < aula02` por UC para reduzir o espaço de procura.
- **Filtragem de domínio por dias**: limitar previamente dias por turma (equivalente a “≤4 dias” como *hard* leve).
- **Consecutividade soft→hard**: encontrar uma solução soft, depois reforçar a consecutividade como *hard* e refinar.
- **Exploração multi-arranque**: diferentes ordens de variáveis para escapar a locais sub-ótimos.


# Conclusões

O resultado da otimização evidenciou claramente a principal limitação do algoritmo Greedy Search.

Embora o algoritmo otimizado tenha sido encontrado em tempo nulo (0s), o Score obtido (41) ficou abaixo do Score da melhor solução da pesquisa Backtracking (47), a solução ótima.  

Se o objetivo da solução for implementar este agente num sistema com capacidades reduzidas, a Greedy Search é a solução mais prática e viável.
Como o algoritmo prioriza sempre o nó  mais favorável (menor valor heurístico), o processo falha em reserver combinações de alto valor nas etapas posteriores, como as aulas consecutivas, limitando a sua otimização global.

A abordagem padrão com a Pesquisa Backtracking, apesar de ser significativamente mais lenta, conseguiu escapar ao problema da Greedy e conseguir criar aulas no mesmo dia em blocos consecutivos. 